### mysql과 python 연동 
1. mysql과 연동을 할 수 있는 기능을 가진 라이브러리 설치 
    - python에서 라이브러리들이 저장되어 있는 공간
    - R에서는 CRAN에 저장소가 있다면 python에서는 pip 저장소에 저장되어있다. 
    - 라이브러리 설치 명령어 : pip install 라이브러리명
    - mysql과 연동하는 라이브러리의 이름은 pymysql
    - pip install pymysql
2. 라이브러리 로드 
    - import 라이브러리명
    - from 라이브러리명 import (Class|function|module)
    - as 별칭 -> 라이브러리를 별칭을 통해 간단하게 쓰기 위해서
3. mysql 서버의 정보를 입력하고 연결
    - DB서버의 주소 : host
        - localhost, 127.0.0.1 : 내 컴퓨터
        - 외부의 주소 (ex : 172.168.1.40)
    - port
        - mysql의 기본포트 : 3306
    - user
        - mysql에 연결할때 사용하는 사용자의 이름
        - 기본 계정 (root)
    - password
        - mysql에 연결할 때 사용하는 사용자의 비밀번호
    - db
        - mysql 서버에서 사용할 Database의 이름
- DB서버와 python공간 사이에 가상 공간을 생성
    - cursor는 가상 공간의 이름
- sql 쿼리문을 이용하여 cursor에 질의를 보낸다. 
    - 데이터의 변환 질의 (insert, update, delete)
        - 질의를 보낸다.(execute())
        - DB서버의 데이터를 변경(commit())
        - DB서버와의 연결을 종료(close())
    - 데이터의 로드 질의(select)
        - 질의를 보낸다(execute())
        - 질의에 대한 결과물을 받아온다.(fetchall())
        - DB서버와의 연결을 종료(close())

In [ ]:
# 라이브러리 설치 
!pip install pymysql

In [ ]:
# 라이브러리 로드 
import pymysql

In [ ]:
## mysql 연결시 cryptography 에러 발생 시 
!pip install cryptography

In [ ]:
_db = pymysql.connect(
    host = 'localhost',     # 172.30.1.55
    port = 3306, 
    user = 'root',          # ubion
    password = '1234', 
    db = 'ubion1'
)

In [ ]:
# cursor 생성 (기본값 설정) 
cursor = _db.cursor()
# cursor2 생성 (옵션 변경)
cursor2 = _db.cursor(pymysql.cursors.DictCursor)

# cursor의 기본 설정대로 생성시 데이터를 로드하는 과정에서 받아오는 
# 데이터의 타입이 2차원 tuple
# DictCursor 이용시 받아오는 데이터의 타입은 list안에 dict 형태

In [ ]:
# query문을 작성 -> 문자형 데이터
query = """
    select 
    * 
    from 
    tran_1
"""
# execute()를 이용하여 query문을 cursor에게 보낸다.
cursor.execute(query)
cursor2.execute(query)

In [ ]:
cursor.fetchall()

In [ ]:
cursor2.fetchall()

In [ ]:
# DB 서버와 연결을 종료
_db.close()

In [ ]:
## DB서버에 데이터들 수정 
query = """
    update 
    tran_1 
    set 
    price = 10000
    where 
    transaction_id = 'T0000000113'
"""

In [ ]:
## 데이터 변환시 흐름 : execute() -> commit()
cursor.execute(query)

In [ ]:
select_query = """
    select * from tran_1 limit 5
"""
cursor.execute(select_query)
cursor.fetchall()

In [ ]:
## cursor에서 변경한 내역을 DB 서버에 보낸다. (동기화)
_db.commit()

In [ ]:
## select문을 사용할때의 함수 생성 
def query_select(_sql):
    # cursor에 _sql의 질의를 보낸다. 
    cursor.execute(_sql)
    # 결과물을 돌려받는다. 
    result = cursor.fetchall()
    return result

In [ ]:
select1 = """
    select * from tran_1 limit 5
"""
query_select(select1)

In [ ]:
## insert, update, delete문을 이용할때 사용할 함수 생성
def cud_query(_sql):
    # _sql를 cursor에 execute()
    cursor.execute(_sql)
    # DB서버와 동기화 
    _db.commit()
    return 'Query OK'

In [ ]:
delete1 = """
    delete 
    from 
    tran_1
    where 
    transaction_id = 'T0000000113'
"""

In [ ]:
cud_query(delete1)

In [ ]:
query = """
    DELETE from tran_1
"""

# query 문자열이 select로 시작하는가?
# case1 (문자를 잘라서 확인)
print(query.strip()[0:6].lower() == 'select')
# case2 (공백을 기준으로 문자열을 나눠준다.)
print(query.split()[0].lower() == 'select')
# case3 (문자의 시작이 특정 단어로 시작하는가?)
print(query.lower().strip().startswith('select'))


In [ ]:
### query_select()함수와 cud_query() 함수를 결합하여 하나의 함수를 생성
def sql_query(_sql, *_values):
    # cursor에 질의를 보낸다. 
    cursor.execute(_sql, _values)
    # _sql에 입력되는 인자의 값이 select문이라면?
    if _sql.lower().strip().startswith('select'):
        # select문일 때 실행 할 코드 작성
        result = cursor.fetchall()
    else:
        # select문이 아닐 때 실행 할 코드 작성 
        _db.commit()
        result = 'Query OK'
    return result

In [ ]:
select2 = """
    select * from tran_1 where price > 20000
"""
sql_query(select2)

In [ ]:
delete2 = """
    delete from tran_1 where transaction_id = 'T0000000115'
"""
sql_query(delete2)

In [ ]:
# select3 = """
#     select * from tran_1 where price >= %s and price <= %s
# """
select3 = """
    select * from tran_1 where price between %s and %s
"""
value = (50000, 100000)
cursor.execute(select3, value)
cursor.fetchall()

In [ ]:
sql_query(select3, 50000, 80000)

In [ ]:
select4 = """
    select * from tran_1
"""
sql_query(select4)


In [ ]:
## mysql server와의 연동을 하는 Class 선언
class MyDB:
    # 생성자 함수 (DB서버의 정보를 입력)
    def __init__(
            self, 
            _host = 'localhost', 
            _port = 3306, 
            _user = 'root', 
            _pw = '1234', 
            _db = 'ubion'
    ):
        self.host = _host
        self.port = _port
        self.user = _user
        self.pw = _pw
        self.db = _db
    def sql_query(self, _sql, *_values):
        # DB서버와의 연결
        mydb = pymysql.connect(
            host = self.host, 
            port = self.port, 
            user = self.user, 
            password = self.pw, 
            db = self.db
        )
        # cursor 생성
        cursor = mydb.cursor(pymysql.cursors.DictCursor)
        # _sql, _values를 이용하여 cursor에 질의를 던진다. 
        cursor.execute(_sql, _values)
        # _sql이 select문인가 확인 
        if _sql.lower().strip().startswith('select'):
            # select문이면 
            result = cursor.fetchall()
        else:
            # selec문이 아니라면
            mydb.commit()
            result = 'Query OK'
        # DB 서버와의 연결을 종료
        mydb.close()
        # 결과값을 되돌려준다. 
        return result
        

In [ ]:
# MyDB class 생성
db1 = MyDB()

In [ ]:
# db1 : 서버의 정보가 내 컴퓨터
select1 = """
    select * from tran_1 limit %s
"""
db1.sql_query(select1, 3)

In [62]:
## 외부의 컴퓨터 접속 
db2 = MyDB(
    _host = '172.30.1.55', 
    _port = 3306, 
    _user = 'ubion', 
    _pw = '1234', 
    _db = 'ubion'
)

In [63]:
select1 = """
    select * from corona limit 5
"""
db2.sql_query(select1)

[{'MyUnknownColumn': 0,
  'createDt': '2022-06-08 09:09:05.982',
  'deathCnt': 24305,
  'decideCnt': 18188200,
  'seq': 904,
  'stateDt': 20220608,
  'stateTime': '00:00',
  'updateDt': None,
  'accExamCnt': '',
  'accDefRate': ''},
 {'MyUnknownColumn': 1,
  'createDt': '2022-06-07 09:09:00.897',
  'deathCnt': 24299,
  'decideCnt': 18174842,
  'seq': 903,
  'stateDt': 20220607,
  'stateTime': '00:00',
  'updateDt': '2022-06-08 09:10:36.846',
  'accExamCnt': '',
  'accDefRate': ''},
 {'MyUnknownColumn': 2,
  'createDt': '2022-06-06 09:00:06.734',
  'deathCnt': 24279,
  'decideCnt': 18168670,
  'seq': 902,
  'stateDt': 20220606,
  'stateTime': '00:00',
  'updateDt': '2022-06-08 09:10:50.441',
  'accExamCnt': '',
  'accDefRate': ''},
 {'MyUnknownColumn': 3,
  'createDt': '2022-06-05 08:53:19.426',
  'deathCnt': 24258,
  'decideCnt': 18163648,
  'seq': 901,
  'stateDt': 20220605,
  'stateTime': '00:00',
  'updateDt': '2022-06-08 09:11:04.758',
  'accExamCnt': '',
  'accDefRate': ''},
 {'My